# Aim

In this notebook, we're focusing on the relationship between non-response (for GI question) and the % of Non-English speakers in our LA's.

* We will explore at the Shannon index, which is a measure of religious diversity within a community.
* We will calculate the SI for our different LAs. 
* A higher Shannon Index indicates greater religious diversity.

This will allow us to make some inferences about the religious diversity among our 331 LAs.


# Religion

## Import  libraries

In [1]:
# used to manipulate dataframes
import pandas as pd

# used to create visualisations
import matplotlib.pylab as plt

# used to calculate shannon index
import numpy as np

# used to create interactive visualisations
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColorBar, BasicTicker, PrintfTickFormatter
from bokeh.models import LinearColorMapper

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Read-in data

We will read in the original religion dataset.

In [2]:
rel = pd.read_excel('../Data/religion_gi.xlsx')

In [3]:
# Let's take a brief look..

rel.head()

,Lower tier local authorities Code,Lower tier local authorities,Gender identity (4 categories) Code,Gender identity (4 categories),Religion (10 categories) Code,Religion (10 categories),Observation
0,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,0
1,E06000001,Hartlepool,-8,Does not apply,1,No religion,0
2,E06000001,Hartlepool,-8,Does not apply,2,Christian,0
3,E06000001,Hartlepool,-8,Does not apply,3,Buddhist,0
4,E06000001,Hartlepool,-8,Does not apply,4,Hindu,0


## Cleaning data

In [5]:
# Let's rename our columns and give them less clunky names

rel.rename(columns={'Lower tier local authorities Code': 'LA_code', 'Lower tier local authorities': 'LA_name', 'Gender identity (4 categories) Code': 'GI_code', 'Gender identity (4 categories)': 'GI_categories', 'Religion (10 categories) Code': 'Religion_code', 'Religion (10 categories)': 'Religion_categories'}, inplace=True)

In [6]:
# Subset data - we want to focus on non-response and exclude 'Does not apply'

non_resp = rel[(rel.GI_code == 3) & (rel.GI_code != -8) & (rel.Religion_code != -8) & (rel.Religion_code != 9)]

In [7]:
non_resp.head()

,LA_code,LA_name,GI_code,GI_categories,Religion_code,Religion_categories,Observation
31,E06000001,Hartlepool,3,Not answered,1,No religion,849
32,E06000001,Hartlepool,3,Not answered,2,Christian,1359
33,E06000001,Hartlepool,3,Not answered,3,Buddhist,13
34,E06000001,Hartlepool,3,Not answered,4,Hindu,34
35,E06000001,Hartlepool,3,Not answered,5,Jewish,1


# Pre-processing

Now we're going to create the Shannon index. These are the steps:

1) Calculate the religion proportions for each LA
2) Execute SI formula - take proportion of each religion and its natural logarithm
3) Sum those values for each LA

Then we will reduce the dataset so that it just lists the shannon index for each unique LA.
Finally, we'll create a colormap object which will allow us to set the colour mapping of datapoints.

## Calculations

In [8]:
# Created an empty column to hold proportions of religions within each LA

non_resp['Proportions'] = ''

for i in non_resp.LA_name.unique():
    
    b = non_resp[non_resp.LA_name == i]
    
    prop = b.Observation / b.Observation.sum()
    
    non_resp.loc[b.index, 'Proportions'] = round(prop, 4)

    

/var/folders/dm/468m4xf52c71yhldz5ck50pw0000gn/T/ipykernel_4423/3263719366.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_resp['Proportions'] = ''


In [9]:
non_resp.head()

,LA_code,LA_name,GI_code,GI_categories,Religion_code,Religion_categories,Observation,Proportions
31,E06000001,Hartlepool,3,Not answered,1,No religion,849,0.3564
32,E06000001,Hartlepool,3,Not answered,2,Christian,1359,0.5705
33,E06000001,Hartlepool,3,Not answered,3,Buddhist,13,0.0055
34,E06000001,Hartlepool,3,Not answered,4,Hindu,34,0.0143
35,E06000001,Hartlepool,3,Not answered,5,Jewish,1,0.0004
36,E06000001,Hartlepool,3,Not answered,6,Muslim,96,0.0403
37,E06000001,Hartlepool,3,Not answered,7,Sikh,11,0.0046
38,E06000001,Hartlepool,3,Not answered,8,Other religion,19,0.008
71,E06000002,Middlesbrough,3,Not answered,1,No religion,1398,0.3189
72,E06000002,Middlesbrough,3,Not answered,2,Christian,2028,0.4626


In [10]:
# Converted proportions column to numeric type

non_resp['Proportions'] = pd.to_numeric(non_resp['Proportions'])


# Calculated the intermediate value used in the Shannon Index
# Formula for SI involves the proportion of each religion and the natural logarithm of that proportion

non_resp['Calc'] = np.where(non_resp['Proportions'] > 0, - non_resp['Proportions'] * np.log(non_resp['Proportions']), 0)



/var/folders/dm/468m4xf52c71yhldz5ck50pw0000gn/T/ipykernel_4423/4126200099.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_resp['Proportions'] = pd.to_numeric(non_resp['Proportions'])
/opt/anaconda3/envs/newenv/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/dm/468m4xf52c71yhldz5ck50pw0000gn/T/ipykernel_4423/4126200099.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_r

In [11]:
# Created an empty column titled Shannon_idx

non_resp['Shannon_idx'] = ''

for i in non_resp.LA_code.unique():
    
    b = non_resp[non_resp.LA_code == i]
    
    # used to calculate shannon index - we sum each value in the calc column 
    summed = sum(b.Calc)
    
    non_resp.loc[b.index, 'Shannon_idx'] = summed

/var/folders/dm/468m4xf52c71yhldz5ck50pw0000gn/T/ipykernel_4423/1860448477.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_resp['Shannon_idx'] = ''


In [12]:
# Success.

non_resp

,LA_code,LA_name,GI_code,GI_categories,Religion_code,Religion_categories,Observation,Proportions,Calc,Shannon_idx
31,E06000001,Hartlepool,3,Not answered,1,No religion,849,0.3564,0.367698,0.973174
32,E06000001,Hartlepool,3,Not answered,2,Christian,1359,0.5705,0.320189,0.973174
33,E06000001,Hartlepool,3,Not answered,3,Buddhist,13,0.0055,0.028617,0.973174
34,E06000001,Hartlepool,3,Not answered,4,Hindu,34,0.0143,0.060739,0.973174
35,E06000001,Hartlepool,3,Not answered,5,Jewish,1,0.0004,0.003130,0.973174
...,...,...,...,...,...,...,...,...,...,...
13234,W06000024,Merthyr Tydfil,3,Not answered,4,Hindu,7,0.0042,0.022985,0.844808
13235,W06000024,Merthyr Tydfil,3,Not answered,5,Jewish,0,0.0000,0.000000,0.844808
13236,W06000024,Merthyr Tydfil,3,Not answered,6,Muslim,16,0.0095,0.044236,0.844808
13237,W06000024,Merthyr Tydfil,3,Not answered,7,Sikh,6,0.0036,0.020257,0.844808


In [13]:
# Reduce dataset so that it just contains each unique LA and its SI

unique_shannon_df = non_resp[['LA_name', 'Shannon_idx']].drop_duplicates(subset=['LA_name'])

# Let's take a look at our top 5 most diverse LAs

unique_shannon_df.sort_values(by = 'Shannon_idx', ascending = False).head()

,LA_name,Shannon_idx
11711,Hillingdon,1.620787
11151,Barnet,1.604286
11751,Hounslow,1.60157
11391,Ealing,1.591577
6271,Oadby and Wigston,1.575674


In [14]:
# Sort LA_name by alphabetical order and reset the index

unique_shannon_df = unique_shannon_df.sort_values(by = 'LA_name').reset_index(drop = True)

In [15]:
unique_shannon_df.head()

,LA_name,Shannon_idx
0,Adur,0.881446
1,Allerdale,0.78292
2,Amber Valley,0.823553
3,Arun,0.878272
4,Ashfield,0.900455


In [16]:
# Before we map the shannon index, we should normalise the values
# This will make it easier to identify trends or patterns in the data

unique_shannon_df['Normalised'] =  (unique_shannon_df['Shannon_idx'] - unique_shannon_df['Shannon_idx'].min()) / (unique_shannon_df['Shannon_idx'].max() - unique_shannon_df['Shannon_idx'].min())


## Color mapping in Bokeh

Bokeh is the interactive visualisation library that we will be using, so we'll have to create our colour mapping object using its functions.

In [17]:
# Created colour map object in Bokeh
# Viridis256 chosen because its good at representing continuous variables

color_map = LinearColorMapper(palette="Viridis256", low=unique_shannon_df.Normalised.min(), high=unique_shannon_df.Normalised.max())

# Success. 

color_map

LinearColorMapper(id='p1001', ...)

## Read-in merged dataframe

This merged_df was created in the 'Main_Lang_NR_GI.ipynb' notebook, and contains the modified main language dataset, along with additional columns on 'region' and 'urban-rural'.

In [18]:
merged_df = pd.read_csv('../Data/lang_rural_region_gi.csv')

In [19]:
# Let's check it out

merged_df.head()

,LA_name,Observation,Non_Eng_Percentages,NR_rate,region,Urb_Rur
0,Adur,1971,3.14,4.68,South East,Predominantly Urban
1,Allerdale,1073,1.15,4.61,North West,Predominantly Rural
2,Amber Valley,1850,1.51,5.44,East Midlands,Predominantly Urban
3,Arun,9469,5.89,5.44,South East,Predominantly Urban
4,Ashfield,3944,3.22,5.64,East Midlands,Predominantly Urban


In [20]:
# Cool. All we need to do is append our Normalised column to this dataset!

merged_df['Shannon_idx'] = unique_shannon_df['Normalised']

In [21]:
merged_df.head()

,LA_name,Observation,Non_Eng_Percentages,NR_rate,region,Urb_Rur,Shannon_idx
0,Adur,1971,3.14,4.68,South East,Predominantly Urban,0.205076
1,Allerdale,1073,1.15,4.61,North West,Predominantly Rural,0.099143
2,Amber Valley,1850,1.51,5.44,East Midlands,Predominantly Urban,0.142831
3,Arun,9469,5.89,5.44,South East,Predominantly Urban,0.201664
4,Ashfield,3944,3.22,5.64,East Midlands,Predominantly Urban,0.225514


# Data processing

Let's now create a standalone Bokeh plot and colour our data points by their Shannon index.

## Visualisation - Bokeh

In [22]:

# Bokeh has a hover tool, allowing you to scroll over dps to reveal info
# To configure the tool, we must set our tooltips arguments...

# We simply define a list of tuples which refer to column values in our final_df 

tool = [
    ("index", "$index"),
    ("(x,y)", "(@Non_Eng_Percentages, @NR_rate)"),
    ("name","@LA_name"),
    ("Shannon_idx", "@Shannon_idx")
]


# Create graph figure, set title and x and y labels

p2 = figure(title="Relationship between Non-response Rate and Non-English Speakers", x_axis_label="Non-response Rate", y_axis_label="Percentage of Non-English Speakers", tooltips = tool)


# Create scatterplot for x and y column

p2.scatter("Non_Eng_Percentages", "NR_rate", source = merged_df, fill_alpha = 0.5, size = 10,  color={'field': 'Shannon_idx', 'transform': color_map})


# Create colour bar and set the color_mapper parameter to 'color_map' which we produced earlier

color_bar = ColorBar(color_mapper=color_map,
                     title='Shannon Index',
                     ticker=BasicTicker(desired_num_ticks=5),
                     formatter=PrintfTickFormatter(format='%.2f'))

# Add the color bar to the plot
p2.add_layout(color_bar, 'right')

# Display output 

output_notebook()
show(p2)

Loading BokehJS ...

# Outputs

Now that we've added on the shannon index column to our language dataset, let's save this to a csv. Our aim will now be to create an interactive Bokeh plot for GI which has drop downs allowing users to switch between colouring dps by:

* region
* urban-rural classification
* shannon index

In [23]:
merged_df.to_csv('../Data/final_lang_gi.csv', index = False)

# Conclusion

We have now finished colouring our data points by their Shannon index. This means our final 'merged_df' can be used to create an interactive scatterplot with drop-downs, where users can select to colour data points by the following:

* region
* urban-rural classification
* shannon index

If you'd like to see how we manage that, please proceed to [GI_Outputs.ipynb](./GI_Outputs.ipynb).
However, before you do this you might want to check out [Religion_2_GI.ipynb](./Religion_2_GI.ipynb) where we use the same religion dataset to explore a new relationship: the relationship between the % of religious group and their contribution to non-response rates in LAs. This will be the second set of scatterplots in our final application to be hosted on Heroku.